In [1]:
import time
from pylab import *
import matplotlib.gridspec as gridspec

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
print(torch.cuda.is_available())

In [3]:
content_path = 'data'

In [ ]:
pip install h5py

In [ ]:
import h5py

def inspect_hdf5_file(file_path):
    with h5py.File(file_path, 'r') as f:
        # Print all root level groups and datasets
        def print_structure(name, obj):
            if isinstance(obj, h5py.Dataset):
                print("1")
                print(f"Dataset: {name}, Shape: {obj.shape}, Dtype: {obj.dtype}")
            elif isinstance(obj, h5py.Group):
                print("2")
                print(f"Group: {name}")

        f.visititems(print_structure)
data_path = content_path+'/2D_CFD_Rand_M0.1_Eta0.01_Zeta0.01_periodic_128_Train.hdf5'
# Example usage
inspect_hdf5_file(data_path)


In [ ]:
import h5py
import numpy as np
import os

def get_norm_val(file_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)


    with h5py.File(file_path, 'r') as f:
        # Load the datasets
        Vx = f['Vx']
        Vy = f['Vy']
        Vx_max = -999
        Vx_min = 999
        Vy_max = -999
        Vy_min = 999



        num_sequences = Vx.shape[0]
        for i in range(num_sequences):
            Vx_seq = Vx[i].reshape(21, 1, 128, 128)

            Vy_seq = Vy[i].reshape(21, 1, 128, 128)

            Vx_max = max(Vx_max, np.max(Vx_seq))
            Vx_min = min(Vx_min, np.min(Vx_seq))
            Vy_max = max(Vy_max, np.max(Vy_seq))
            Vy_min = min(Vy_min, np.min(Vy_seq))

            # Save the combined sequence as a .npy file
            #np.save(os.path.join(output_dir, f'sequence_{i}.npy'), combined_seq)
        return Vx_max, Vx_min, Vy_max, Vy_min

# Example usage

output_dir = content_path+'/CFD'
Vx_max, Vx_min, Vy_max, Vy_min = get_norm_val(data_path, output_dir)



In [ ]:
import numpy as np
import os


def save_in_npy(file_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)


    with h5py.File(file_path, 'r') as f:
        # Load the datasets
        Vx = f['Vx']
        Vy = f['Vy']
        num_sequences = Vx.shape[0]
        for i in range(num_sequences):
            Vx_seq = Vx[i].reshape(21, 1, 128, 128)
            Vy_seq = Vy[i].reshape(21, 1, 128, 128)
            Vx_seq = (Vx_seq - np.min(Vx_seq)) / (np.max(Vx_seq) - np.min(Vx_seq))
            Vy_seq = (Vy_seq - np.min(Vy_seq)) / (np.max(Vy_seq) - np.min(Vy_seq))
            combined_seq = np.concatenate([Vx_seq, Vy_seq], axis=1)
            np.save(os.path.join(output_dir, f'sequence_{i}.npy'), combined_seq)
        print("save done")

# Example usage

output_dir = content_path+'/CFD_2_channels'
save_in_npy(data_path, output_dir)

In [ ]:
import os
import torch
from torch.utils.data import Dataset
import numpy as np
import re
# import time

class NPYDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        """
        Args:
            data_dir (string): Directory with all the .npy files.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.data_dir = data_dir
        self.transform = transform
        self.file_names = sorted([f for f in os.listdir(data_dir) if f.endswith('.npy')],
                                 key=lambda x: int(re.findall(r'\d+', x)[0]))

        print("len file_names",self.file_names)
        self.num_frames_per_file = 21  # Number of frames per sequence

    def __len__(self):
        # Total number of samples (frames) across all files
        return len(self.file_names) * self.num_frames_per_file

    def __getitem__(self, idx):
        #print("asdsa")
        # t1 = time.time()
        # Calculate the corresponding file index and frame index
        # Find the corresponding file and frame index within that file
        file_idx = idx // self.num_frames_per_file
        frame_idx = idx % self.num_frames_per_file
        
        

        # Load the corresponding .npy file
        npy_file = os.path.join(self.data_dir, self.file_names[file_idx])
        sequence = np.load(npy_file, mmap_mode='r') # Load the entire sequence from the .npy file
       
        # Extract the specific frame from the sequence
        frame = sequence[frame_idx]  # Shape: (4, 128, 128)

        if self.transform:
            frame = self.transform(frame)
        # t4 = time.time()
        


        return torch.tensor(frame, dtype=torch.float32)

                 






# Example usage
output_dir = content_path+'/CFD_2_channels'
print(output_dir)
dataset = NPYDataset(data_dir=output_dir)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
print("len dataloader",len(dataloader))
print("len dataset",len(dataset))




In [8]:
import torch
import torch.nn.functional as F
import math


class KANLinear(torch.nn.Module):
    def __init__(
        self,
        in_features,
        out_features,
        grid_size=5,
        spline_order=3,
        scale_noise=0.1,
        scale_base=1.0,
        scale_spline=1.0,
        enable_standalone_scale_spline=True,
        base_activation=torch.nn.SiLU,
        grid_eps=0.02,
        grid_range=[-1, 1],
    ):
        super(KANLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.grid_size = grid_size
        self.spline_order = spline_order

        h = (grid_range[1] - grid_range[0]) / grid_size
        grid = (
            (
                torch.arange(-spline_order, grid_size + spline_order + 1) * h
                + grid_range[0]
            )
            .expand(in_features, -1)
            .contiguous()
        )
        self.register_buffer("grid", grid)

        self.base_weight = torch.nn.Parameter(torch.Tensor(out_features, in_features))
        self.spline_weight = torch.nn.Parameter(
            torch.Tensor(out_features, in_features, grid_size + spline_order)
        )
        if enable_standalone_scale_spline:
            self.spline_scaler = torch.nn.Parameter(
                torch.Tensor(out_features, in_features)
            )

        self.scale_noise = scale_noise
        self.scale_base = scale_base
        self.scale_spline = scale_spline
        self.enable_standalone_scale_spline = enable_standalone_scale_spline
        self.base_activation = base_activation()
        self.grid_eps = grid_eps

        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.kaiming_uniform_(self.base_weight, a=math.sqrt(5) * self.scale_base)
        with torch.no_grad():
            noise = (
                (
                    torch.rand(self.grid_size + 1, self.in_features, self.out_features)
                    - 1 / 2
                )
                * self.scale_noise
                / self.grid_size
            )
            self.spline_weight.data.copy_(
                (self.scale_spline if not self.enable_standalone_scale_spline else 1.0)
                * self.curve2coeff(
                    self.grid.T[self.spline_order : -self.spline_order],
                    noise,
                )
            )
            if self.enable_standalone_scale_spline:
                # torch.nn.init.constant_(self.spline_scaler, self.scale_spline)
                torch.nn.init.kaiming_uniform_(self.spline_scaler, a=math.sqrt(5) * self.scale_spline)

    def b_splines(self, x: torch.Tensor):
        """
        Compute the B-spline bases for the given input tensor.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, in_features).

        Returns:
            torch.Tensor: B-spline bases tensor of shape (batch_size, in_features, grid_size + spline_order).
        """
        assert x.dim() == 2 and x.size(1) == self.in_features

        grid: torch.Tensor = (
            self.grid
        )  # (in_features, grid_size + 2 * spline_order + 1)
        x = x.unsqueeze(-1)
        bases = ((x >= grid[:, :-1]) & (x < grid[:, 1:])).to(x.dtype)
        for k in range(1, self.spline_order + 1):
            bases = (
                (x - grid[:, : -(k + 1)])
                / (grid[:, k:-1] - grid[:, : -(k + 1)])
                * bases[:, :, :-1]
            ) + (
                (grid[:, k + 1 :] - x)
                / (grid[:, k + 1 :] - grid[:, 1:(-k)])
                * bases[:, :, 1:]
            )

        assert bases.size() == (
            x.size(0),
            self.in_features,
            self.grid_size + self.spline_order,
        )
        return bases.contiguous()

    def curve2coeff(self, x: torch.Tensor, y: torch.Tensor):
        """
        Compute the coefficients of the curve that interpolates the given points.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, in_features).
            y (torch.Tensor): Output tensor of shape (batch_size, in_features, out_features).

        Returns:
            torch.Tensor: Coefficients tensor of shape (out_features, in_features, grid_size + spline_order).
        """
        assert x.dim() == 2 and x.size(1) == self.in_features
        assert y.size() == (x.size(0), self.in_features, self.out_features)

        A = self.b_splines(x).transpose(
            0, 1
        )  # (in_features, batch_size, grid_size + spline_order)
        B = y.transpose(0, 1)  # (in_features, batch_size, out_features)
        solution = torch.linalg.lstsq(
            A, B
        ).solution  # (in_features, grid_size + spline_order, out_features)
        result = solution.permute(
            2, 0, 1
        )  # (out_features, in_features, grid_size + spline_order)

        assert result.size() == (
            self.out_features,
            self.in_features,
            self.grid_size + self.spline_order,
        )
        return result.contiguous()

    @property
    def scaled_spline_weight(self):
        return self.spline_weight * (
            self.spline_scaler.unsqueeze(-1)
            if self.enable_standalone_scale_spline
            else 1.0
        )

    def forward(self, x: torch.Tensor):
        assert x.size(-1) == self.in_features
        original_shape = x.shape
        x = x.reshape(-1, self.in_features)

        base_output = F.linear(self.base_activation(x), self.base_weight)
        spline_output = F.linear(
            self.b_splines(x).view(x.size(0), -1),
            self.scaled_spline_weight.view(self.out_features, -1),
        )
        output = base_output + spline_output

        output = output.reshape(*original_shape[:-1], self.out_features)
        return output

    @torch.no_grad()
    def update_grid(self, x: torch.Tensor, margin=0.01):
        assert x.dim() == 2 and x.size(1) == self.in_features
        batch = x.size(0)

        splines = self.b_splines(x)  # (batch, in, coeff)
        splines = splines.permute(1, 0, 2)  # (in, batch, coeff)
        orig_coeff = self.scaled_spline_weight  # (out, in, coeff)
        orig_coeff = orig_coeff.permute(1, 2, 0)  # (in, coeff, out)
        unreduced_spline_output = torch.bmm(splines, orig_coeff)  # (in, batch, out)
        unreduced_spline_output = unreduced_spline_output.permute(
            1, 0, 2
        )  # (batch, in, out)

        # sort each channel individually to collect data distribution
        x_sorted = torch.sort(x, dim=0)[0]
        grid_adaptive = x_sorted[
            torch.linspace(
                0, batch - 1, self.grid_size + 1, dtype=torch.int64, device=x.device
            )
        ]

        uniform_step = (x_sorted[-1] - x_sorted[0] + 2 * margin) / self.grid_size
        grid_uniform = (
            torch.arange(
                self.grid_size + 1, dtype=torch.float32, device=x.device
            ).unsqueeze(1)
            * uniform_step
            + x_sorted[0]
            - margin
        )

        grid = self.grid_eps * grid_uniform + (1 - self.grid_eps) * grid_adaptive
        grid = torch.concatenate(
            [
                grid[:1]
                - uniform_step
                * torch.arange(self.spline_order, 0, -1, device=x.device).unsqueeze(1),
                grid,
                grid[-1:]
                + uniform_step
                * torch.arange(1, self.spline_order + 1, device=x.device).unsqueeze(1),
            ],
            dim=0,
        )

        self.grid.copy_(grid.T)
        self.spline_weight.data.copy_(self.curve2coeff(x, unreduced_spline_output))

    def regularization_loss(self, regularize_activation=1.0, regularize_entropy=1.0):
        """
        Compute the regularization loss.

        This is a dumb simulation of the original L1 regularization as stated in the
        paper, since the original one requires computing absolutes and entropy from the
        expanded (batch, in_features, out_features) intermediate tensor, which is hidden
        behind the F.linear function if we want an memory efficient implementation.

        The L1 regularization is now computed as mean absolute value of the spline
        weights. The authors implementation also includes this term in addition to the
        sample-based regularization.
        """
        l1_fake = self.spline_weight.abs().mean(-1)
        regularization_loss_activation = l1_fake.sum()
        p = l1_fake / regularization_loss_activation
        regularization_loss_entropy = -torch.sum(p * p.log())
        return (
            regularize_activation * regularization_loss_activation
            + regularize_entropy * regularization_loss_entropy
        )


class KAN(torch.nn.Module):
    def __init__(
        self,
        layers_hidden,
        grid_size=5,
        spline_order=3,
        scale_noise=0.1,
        scale_base=1.0,
        scale_spline=1.0,
        base_activation=torch.nn.SiLU,
        grid_eps=0.02,
        grid_range=[-1, 1],
    ):
        super(KAN, self).__init__()
        self.grid_size = grid_size
        self.spline_order = spline_order

        self.layers = torch.nn.ModuleList()
        for in_features, out_features in zip(layers_hidden, layers_hidden[1:]):
            self.layers.append(
                KANLinear(
                    in_features,
                    out_features,
                    grid_size=grid_size,
                    spline_order=spline_order,
                    scale_noise=scale_noise,
                    scale_base=scale_base,
                    scale_spline=scale_spline,
                    base_activation=base_activation,
                    grid_eps=grid_eps,
                    grid_range=grid_range,
                )
            )

    def forward(self, x: torch.Tensor, update_grid=False):
        for layer in self.layers:
            if update_grid:
                layer.update_grid(x)
            x = layer(x)
        return x

    def regularization_loss(self, regularize_activation=1.0, regularize_entropy=1.0):
        return sum(
            layer.regularization_loss(regularize_activation, regularize_entropy)
            for layer in self.layers
        )

In [9]:
import torch
import torch.nn as nn

class PowerfulAutoencoder(nn.Module):
    def __init__(self, latent_dim):
        super(PowerfulAutoencoder, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(2, 32, kernel_size=4, stride=2, padding=1),  # (128, 128, 4) -> (64, 64, 32)
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1), # (64, 64, 32) -> (32, 32, 64)
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1), # (32, 32, 64) -> (16, 16, 128)
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1), # (16, 16, 128) -> (8, 8, 256)
            nn.ReLU(),
            nn.Flatten(),
            #nn.Linear(8*8*256, latent_dim)  # (8*8*256) -> (latent_dim)
            KANLinear(8*8*256, latent_dim)

        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 8*8*256),  # (latent_dim) -> (8*8*256)
            nn.Unflatten(1, (256, 8, 8)),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1), # (8, 8, 256) -> (16, 16, 128)
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1), # (16, 16, 128) -> (32, 32, 64)
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1), # (32, 32, 64) -> (64, 64, 32)
            nn.ReLU(),
            nn.ConvTranspose2d(32, 2, kernel_size=4, stride=2, padding=1), # (64, 64, 32) -> (128, 128, 4)
            nn.Sigmoid()  # Use sigmoid if your image pixels are in the range [0, 1]
        )

    def forward(self, x):
        z = self.encoder(x)
        x_reconstructed = self.decoder(z)
        return x_reconstructed,z

def loss_function(reconstructed, original, latent, lambda_reg=0.1):
    reconstruction_loss = nn.MSELoss()(reconstructed, original)
    latent_reg_loss = torch.mean(torch.norm(latent, p=2, dim=1))
    total_loss = reconstruction_loss + lambda_reg * latent_reg_loss
    return total_loss, reconstruction_loss, latent_reg_loss


In [ ]:
from torch.utils.data import DataLoader, TensorDataset, random_split
model_save_path = content_path + '/model/AE_CFD_KAN_2_channels.pth'




# dataset = torch.tensor(dataset, dtype=torch.float32)
print("dataset",dataset)
# Create DataLoader
batch_size = 32

# Split the dataset into training and testing sets (80% training, 20% testing)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,num_workers =4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,num_workers = 4)

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

model = PowerfulAutoencoder(latent_dim=256).to(device)
print(model)


criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 30
lambda_reg = 1e-6
train_losses = []
test_losses = []

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    train_latent_loss = 0
    step = 0
    for data in train_loader:
        data = data.to(device)  # Move data to GPU
      
        # Forward pass
        output,latent = model(data)
        # loss = criterion(output, data)
        loss, reconstruction_loss, latent_reg_loss = loss_function(output,data,latent,lambda_reg=lambda_reg)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_latent_loss += latent_reg_loss.item()
        step+=1

    train_loss /= len(train_loader)
    train_latent_loss /= len(train_loader)
    train_losses.append(train_loss)

    model.eval()
    test_loss = 0
    test_latent_loss = 0
    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)  # Move data to GPU
            #print("input_data.shape",data.shape)

            # Forward pass
            output,latent = model(data)
            # loss = criterion(output, data)
            loss, reconstruction_loss, latent_reg_loss = loss_function(output,data,latent,lambda_reg=lambda_reg)

            test_loss += loss.item()
            test_latent_loss += latent_reg_loss.item()

    test_loss /= len(test_loader)
    test_latent_loss /= len(test_loader)
    test_losses.append(test_loss)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.6f}, Test Loss: {test_loss:.6f},Train Latent loss: {train_latent_loss:.6f}, Test Latent Loss: {test_latent_loss:.6f}')


print('Training complete')
torch.save(model.state_dict(), model_save_path)

# Plot the training and testing losses
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(train_losses)+1), train_losses, label='Train Loss')
plt.plot(range(1, len(train_losses)+1), test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Testing Loss per Epoch')
plt.legend()
plt.show()

In [ ]:
from matplotlib import colors
def visualize_reconstruction(model, data_loader, num_images=5):
    model.eval()
    # data_iter = iter(data_loader)
    count = 0
    for data in data_loader:
        original_data = data.to(device)
        #see results of a batch
        if count==3:
            break
        count+=1
    with torch.no_grad():
        reconstructed_data,_ = model(original_data)
  

    original_data = original_data.cpu().numpy()
    reconstructed_data = reconstructed_data.cpu().numpy()


    fig, axes = plt.subplots(num_images, 4, figsize=(18, num_images * 2))
    for i in range(num_images):
        for j in range(2):
            ax = axes[i, j*2]

            ax.imshow(original_data[i, j], cmap='viridis')

            ax.set_title(f'Original - Channel {j+1}')
            ax.axis('off')

            ax = axes[i, j*2 + 1]
            ax.imshow(reconstructed_data[i, j], cmap='viridis')
            ax.set_title(f'Reconstructed - Channel {j+1}')
            ax.axis('off')

    plt.tight_layout()
    plt.show()
   

# Visualize some reconstructions
model.load_state_dict(torch.load(model_save_path))
visualize_reconstruction(model, test_loader, num_images=10)


In [ ]:
pip install scikit-image


In [ ]:
from skimage.metrics import structural_similarity as ssim
# Define RRMSE and SSIM functions
def rrmse(img1, img2):
    return np.sqrt(np.mean((img1 - img2) ** 2)) / np.sqrt(np.mean(img1 ** 2))

def compute_ssim(img1, img2):
    return ssim(img1, img2, data_range=img2.max() - img2.min(), multichannel=True,win_size=5, channel_axis=-1)

# Compute RRMSE and SSIM for the entire test dataset
def evaluate_model(model, data_loader):
    model.eval()
    rrmse_values = []
    ssim_values = []
    mse_values = []

    with torch.no_grad():
        for data in data_loader:
            data = data.to(device)

            reconstructed_data,_ = model(data)
            original_data = data.cpu().numpy()
            reconstructed_data = reconstructed_data.cpu().numpy()

            for i in range(original_data.shape[0]):

                original_img = original_data[i].transpose(1, 2, 0)

                reconstructed_img = reconstructed_data[i].transpose(1, 2, 0)

                rrmse_value = rrmse(original_img, reconstructed_img)

                ssim_value = compute_ssim(original_img, reconstructed_img)

                mse = np.mean((original_img - reconstructed_img) ** 2)
               

                rrmse_values.append(rrmse_value)
                ssim_values.append(ssim_value)
                mse_values.append(mse)

    mean_rrmse = np.mean(rrmse_values)
    mean_ssim = np.mean(ssim_values)
    mean_mse = np.mean(mse_values)
    print("metric for AE with KAN")
    #print("metric for AE")
    print(f'Mean RRMSE: {mean_rrmse:.6f}')
    print(f'Mean SSIM: {mean_ssim:.6f}')
    print(f'Mean MSE: {mean_mse:.6f}')

    return mean_rrmse, mean_ssim,mean_mse

# Evaluate the model on the test dataset

mean_rrmse, mean_ssim,mean_mse = evaluate_model(model, test_loader)
print(f'Mean RRMSE: {mean_rrmse:.6f}')
print(f'Mean SSIM: {mean_ssim:.6f}')
print(f'Mean MSE: {mean_mse:.6f}')